In [1]:
#Imports
import itertools
import json
from collections import Counter
import pandas as pd
import numpy as np
import os

In [2]:
#Constants
BASE_FOLDER = "TestResults/EvalTest1stRun/"
BASE_START = "ParamOptim_List_"
BASE_MID = "_SnipLen_"
BASE_END = "_Results.jsonl"
SNIPPET_LENS = [5,10,25,50,75,100]


In [3]:
def helperParser(file):
    temp = []
    with open(file, 'r') as f:
        for line in f:
            temp.append(json.loads(line))
    return temp

In [4]:
loaded = []
for i in range(100):
    print()
    loaded.append(helperParser(BASE_FOLDER+BASE_START+str(i)+BASE_MID+'5'+BASE_END))
    loaded.append(helperParser(BASE_FOLDER+BASE_START+str(i)+BASE_MID+'10'+BASE_END))
    loaded.append(helperParser(BASE_FOLDER+BASE_START+str(i)+BASE_MID+'25'+BASE_END))
    loaded.append(helperParser(BASE_FOLDER+BASE_START+str(i)+BASE_MID+'50'+BASE_END))
    loaded.append(helperParser(BASE_FOLDER+BASE_START+str(i)+BASE_MID+'75'+BASE_END))
    loaded.append(helperParser(BASE_FOLDER+BASE_START+str(i)+BASE_MID+'100'+BASE_END))

In [5]:
print(loaded[0])

[{'keylist_id': 0, 'sniplen': '5', 'c': 0.15, 'tol': 1e-06, 'f1': 0.5108868129889762, 'important_feats_7-8': ['herra', 'koira', 'jätti', 'rouva', 'kännykkä', 'sanoi', 'paperi', 'avain', 'neiti', 'yleisö', 'Minä', 'vauva', 'opettaja', 'parataxis', 'Tense=Past', 'eläin', 'kapteeni', 'minä', 'Herra', ';', 'osoitti', 'laatikko', '*', 'root', 'napata', 'ihmetellä', 'kysyi', 'selitti', 'keksiä', 'leikata', 'korkeus', 'sääntö', '2.', 'kartta', 'lapset', 'hauska', 'lahja', 'juna', 'lapsi', 'ehdottaa', 'hypätä', 'totesi', 'salaisuus', 'lukea', 'lentää', 'vocative', '1.', 'salainen', ',', 'harjoitella', 'seura', 'luokka', 'vene', 'täti', 'Siinä', 'Case=Nom', 'kortti', 'kansi', 'Onneksi', 'nummod', 'seitsemän', 'johtaja', 'linna', 'Tässä', 'mietti', 'leikkiä', 'taatusti', 'taitaa', 'kertoi', 'huusi', 'poliisi', 'koulu', 'pyörä', 'ystävä', '-', 'He', 'syödä', 'cm', '!', 'Emme', 'kirje', 'kuiskasi', 'varastaa', 'esitys', 'kummallinen', 'tasku', 'Nyt', 'PronType=Dem', 'InfForm=2', 'pelottaa', 'disco

In [6]:
def helperParser2(temp_list: list[dict], key: str):
    return [x[key] for x in temp_list]

def flattenList(temp_list):
    return [x for y in temp_list for x in y]

In [7]:
df = pd.DataFrame.from_dict(flattenList(loaded))
df['sniplen'] = df['sniplen'].apply(lambda x: int(x))

In [24]:
print(df[(df['c']==120) & (df['sniplen'] == 100)].sort_values('f1'))

      keylist_id  sniplen      c     tol        f1  \
599           24      100  120.0  0.0001  0.817144   
1655          68      100  120.0  0.0001  0.817736   
719           29      100  120.0  0.0001  0.818006   
575           23      100  120.0  0.0001  0.818517   
503           20      100  120.0  0.0001  0.822380   
...          ...      ...    ...     ...       ...   
2183          90      100  120.0  0.0001  0.868694   
1535          63      100  120.0  0.0001  0.869244   
1247          51      100  120.0  0.0001  0.870937   
1175          48      100  120.0  0.0001  0.872680   
1895          78      100  120.0  0.0001  0.878449   

                                    important_feats_7-8  \
599   [eläin, vocative, viimein, opettaja, ystävä, h...   
1655  [eläin, minä, avain, Minä, vocative, syntyä, s...   
719   [eläin, vocative, avain, Minä, InfForm=2, aarr...   
575   [eläin, vocative, InfForm=2, aarre, Minä, herr...   
503   [herra, avain, jätti, äkkiä, eläin, ranta, JA,... 

In [9]:
param_df = df[['keylist_id','sniplen','c','f1']]

In [10]:
keylist_diffs = []
maxss = []
minss = []
for i in SNIPPET_LENS:
    filtered = param_df[(param_df['sniplen'] == i)].sort_values('f1', ascending=False).drop_duplicates('keylist_id')[['f1']]
    mins = np.min(filtered.to_numpy())
    maxs = np.max(filtered.to_numpy())
    maxss.append(maxs)
    minss.append(mins)
    keylist_diffs.append(np.add(maxs, -mins))
print(keylist_diffs)
print(maxss)
print(minss)

[0.04516120212297747, 0.051806198666189296, 0.06190683232468519, 0.06441427737175076, 0.06427646182987257, 0.061304552681131774]
[0.5778260310490303, 0.6511538361733512, 0.7538065752541941, 0.8303496018417326, 0.8639826685063626, 0.8784489812570037]
[0.5326648289260528, 0.5993476375071619, 0.691899742929509, 0.7659353244699818, 0.7997062066764901, 0.8171444285758719]


In [11]:
min_max_diff = []
for i in range(100):
    filtered = param_df[(param_df['keylist_id'] == i)][['sniplen','f1']]
    maxs = np.array([np.max(filtered[filtered['sniplen'] == x]['f1'].to_numpy()) for x in SNIPPET_LENS])
    mins = np.array([np.min(filtered[filtered['sniplen'] == x]['f1'].to_numpy()) for x in SNIPPET_LENS])
    print(np.add(maxs, -mins))

[0.023198   0.06262594 0.17834538 0.30099928 0.36887204 0.40232351]
[0.01547128 0.04475629 0.14793134 0.27180071 0.34892583 0.3954209 ]
[0.01798383 0.0498861  0.15764957 0.28056595 0.33652655 0.37144422]
[0.01250877 0.04321157 0.14479869 0.26188434 0.33338832 0.36859411]
[0.01473405 0.04913316 0.15332143 0.26722695 0.33796295 0.37892261]
[0.01620975 0.04627019 0.14559273 0.25185358 0.29708198 0.3299924 ]
[0.01388682 0.04023406 0.13769732 0.26581613 0.33539674 0.38053198]
[0.01609377 0.04810377 0.1469498  0.26089128 0.33255252 0.37446865]
[0.01812832 0.05581391 0.17024683 0.27918599 0.34240965 0.37869069]
[0.01296932 0.04163382 0.13774481 0.25348331 0.32557862 0.3645762 ]
[0.01774267 0.05195386 0.1557518  0.28738483 0.36029013 0.40622665]
[0.0150746  0.04965266 0.16317689 0.28941937 0.35564625 0.3931842 ]
[0.01868218 0.05431869 0.16062797 0.27326789 0.33428011 0.37167864]
[0.01628648 0.04799143 0.15641491 0.26894989 0.33310928 0.36288849]
[0.00912571 0.03356383 0.12273385 0.23346218 0.3

In [12]:
query = param_df.sort_values('f1', ascending=False).drop_duplicates(['keylist_id','sniplen'])[['sniplen','c']].to_numpy()
tuples = [(x[0], x[1]) for x in query]
print(Counter(tuples))

Counter({(100.0, 120.0): 100, (75.0, 120.0): 100, (50.0, 120.0): 100, (25.0, 120.0): 98, (10.0, 120.0): 85, (5.0, 120.0): 65, (5.0, 60.0): 35, (10.0, 60.0): 15, (25.0, 60.0): 2})


In [13]:
print(param_df.sort_values('f1', ascending=False).drop_duplicates(['keylist_id','sniplen']))

      keylist_id  sniplen      c        f1
1895          78      100  120.0  0.878449
1175          48      100  120.0  0.872680
1247          51      100  120.0  0.870937
1535          63      100  120.0  0.869244
2183          90      100  120.0  0.868694
...          ...      ...    ...       ...
1370          57        5   60.0  0.542394
554           23        5   60.0  0.540912
651           27        5  120.0  0.538875
3              0        5  120.0  0.534085
483           20        5  120.0  0.532665

[600 rows x 4 columns]


In [14]:
print(param_df[param_df.sniplen == 10].sort_values('f1', ascending=False).drop_duplicates(['keylist_id'])[:100])

      keylist_id  sniplen      c        f1
1878          78       10   60.0  0.651154
1230          51       10   60.0  0.648104
439           18       10  120.0  0.645314
367           15       10  120.0  0.645246
1519          63       10  120.0  0.644005
...          ...      ...    ...       ...
559           23       10  120.0  0.610449
655           27       10  120.0  0.609733
7              0       10  120.0  0.609355
1639          68       10  120.0  0.608449
486           20       10   60.0  0.599348

[100 rows x 4 columns]


In [15]:
print(param_df[param_df.sniplen == '25'].sort_values('f1', ascending=False).drop_duplicates(['keylist_id'])[:100])

Empty DataFrame
Columns: [keylist_id, sniplen, c, f1]
Index: []


In [16]:
print(param_df[param_df.sniplen == '50'].sort_values('f1', ascending=False).drop_duplicates(['keylist_id'])[:100])

Empty DataFrame
Columns: [keylist_id, sniplen, c, f1]
Index: []


In [17]:
print(param_df[param_df.sniplen == '75'].sort_values('f1', ascending=False).drop_duplicates(['keylist_id'])[:100])

Empty DataFrame
Columns: [keylist_id, sniplen, c, f1]
Index: []


In [18]:
print(param_df[param_df.sniplen == 100].sort_values('f1', ascending=False).drop_duplicates(['keylist_id'])[:100]['c'].to_numpy())

[120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120.
 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120.
 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120.
 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120.
 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120.
 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120.
 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120. 120.
 120. 120.]


In [19]:
print(df[(df.sniplen == 100) & (df.c==120)].sort_values('f1', ascending=False))

      keylist_id  sniplen      c     tol        f1  \
1895          78      100  120.0  0.0001  0.878449   
1175          48      100  120.0  0.0001  0.872680   
1247          51      100  120.0  0.0001  0.870937   
1535          63      100  120.0  0.0001  0.869244   
2183          90      100  120.0  0.0001  0.868694   
...          ...      ...    ...     ...       ...   
503           20      100  120.0  0.0001  0.822380   
575           23      100  120.0  0.0001  0.818517   
719           29      100  120.0  0.0001  0.818006   
1655          68      100  120.0  0.0001  0.817736   
599           24      100  120.0  0.0001  0.817144   

                                    important_feats_7-8  \
1895  [kännykkä, vocative, jätti, –, aarre, yleisö, ...   
1175  [herra, avain, vocative, tahtoa, lapsi, hauska...   
1247  [kännykkä, sanoi, eläin, jätti, *, vocative, ;...   
1535  [eläin, vocative, lapset, *, hypätä, ranta, sä...   
2183  [vocative, eläin, kännykkä, InfForm=2, avain, ... 

In [20]:
print(Counter([x for y in df.sort_values('f1', ascending=False)[:1]['important_feats_13+'].to_numpy() for x in y]))

Counter({'herttua': 1, 'juuli': 1, 'noita': 1, 'palatsi': 1, 'Derivation=Sti': 1, 'prinssi': 1, 'jotakin': 1, 'acl': 1, 'hius': 1, 'prinsessa': 1, 'mun': 1, 'Mä': 1, 'käveli': 1, 'molemmat': 1, '”': 1, 'mutista': 1, 'Derivation=Llinen,Vs': 1, 'liittyä': 1, 'vanhempi': 1, 'mua': 1, 'minusta': 1, 'lakata': 1, 'tuoksu': 1, 'välittömästi': 1, 'huuli': 1, 'nmod:poss': 1, 'katsoa': 1, 'obj': 1, 'raha': 1, 'Siis': 1, 'sotilas': 1, 'kohta': 1, 'sama': 1, 'pitäisi': 1, 'Derivation=Vs': 1, 'käsi': 1, 'mä': 1, 'raskas': 1, 'puhua': 1, 'suudella': 1, 'minut': 1, 'haluta': 1, 'Mitä': 1, 'hiekka': 1, 'ihminen': 1, 'minua': 1, 'kannattaa': 1, 'väli': 1, 'Derivation=Ton': 1, 'ollut': 1, 'tajuta': 1, 'tavalla': 1, 'siksi': 1, 'vartija': 1, 'veri': 1, 'vähemmän': 1, 'Derivation=Inen,Vs': 1, 'tappaa': 1, 'Reflex=Yes': 1, 'kävellä': 1, 'puoleen': 1, 'mulle': 1, 'kädet': 1, 'pitänyt': 1, 'todellinen': 1, 'perhe': 1, 'ero': 1, 'lisää': 1, 'joten': 1, 'totuus': 1, 'Anteeksi': 1, 'tosin': 1, 'kipu': 1, 'nmod'